<a href="https://colab.research.google.com/github/sanjaymahajan21/ImageSplicingDetection/blob/main/MTech_Final_Project_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#using pretrained network

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
import json


#prepare dataset-1 "Real and Fake Face Detection" "https://www.kaggle.com/datasets/ciplab/real-and-fake-face-detection?resource=download"
train_path = '/content/drive/MyDrive/KaggleRealandFakeFaceDetection/train'
valid_path = '/content/drive/MyDrive/KaggleRealandFakeFaceDetection/valid'
test_path = '/content/drive/MyDrive/KaggleRealandFakeFaceDetection/test'


train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10)


Found 1633 images belonging to 2 classes.
Found 306 images belonging to 2 classes.
Found 102 images belonging to 2 classes.


In [ ]:
  # Training module using ResNet50

  input_tensor = tf.keras.layers.Input(shape=(224,224,3))

  base_model = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

  for layer in base_model.layers:
      layer.trainable = False

  x = base_model.output
  x = Flatten()(x)

  x = Dense(1024, activation='relu')(x)
  x = Dense(1024, activation='relu')(x)
  x = Dense(512, activation='relu')(x)
  x = Dense(512, activation='relu')(x)
  x = Dense(256, activation='relu')(x)

  predictions = Dense(1, activation='sigmoid')(x)

  model1 = Model(inputs=base_model.input, outputs=predictions)

  opt = SGD(lr=1e-4, momentum=0.9) #SGD: It stands for Stochastic Gradient Descent, which is the optimization algorithm being used.
  #lr=1e-4: It specifies the learning rate for the optimizer. The learning rate determines the step size at which the optimizer adjusts the model's parameters during training. In this case, the learning rate is set to 10^(-4), which is a small value indicating cautious and slower updates.
  #momentum=0.9: Momentum is a hyperparameter that affects the convergence speed and behavior of the optimizer. It helps accelerate gradient descent in the relevant direction and dampens oscillations. A momentum value of 0.9 means that the optimizer takes into account 90% of the previously accumulated gradients to influence the current update.


  callback_list=[EarlyStopping(monitor="val_loss",patience=100),ModelCheckpoint(filepath="/content/drive/MyDrive/resNet50TEST-OUTPUT.h5",monitor="val_loss",save_best_only=True,verbose=1)]


  model1.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
  #model1.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", false_acceptance_rate, false_rejection_rate, half_total_error_rate])
  model1.summary()


  history = model1.fit_generator(train_batches,validation_data=valid_batches,epochs=30,verbose=1,callbacks=callback_list)
  print("Number of CNN layers:", len(base_model.layers))

  #STORING HISTORY OF TRAINING FOR LATER USE

  with open("history.json", "w") as f:
      json.dump(history.history, f)

94765736/94765736 [==============================] - 5s 0us/step


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

<ipython-input-4-91017a53fcce>:36: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model1.fit_generator(train_batches,validation_data=valid_batches,epochs=30,verbose=1,callbacks=callback_list)


Epoch 1/30
164/164 [==============================] - ETA: 0s - loss: 0.7120 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.69326, saving model to /content/drive/MyDrive/resNet50TEST-OUTPUT.h5
164/164 [==============================] - 677s 4s/step - loss: 0.7120 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/30
164/164 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.5000
Epoch 2: val_loss did not improve from 0.69326
164/164 [==============================] - 32s 198ms/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/30
164/164 [==============================] - ETA: 0s - loss: 0.6936 - accuracy: 0.5000
Epoch 3: val_loss did not improve from 0.69326
164/164 [==============================] - 31s 186ms/step - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 4/30
164/164 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.5000
Epoch 4: val

In [ ]:
# Testing module - using Resnet50 architecture

import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

def load_model_evaluate(model_path, test_data_generator):
    # Load the model
    model = tf.keras.models.load_model(model_path)

    # Generate predictions
    test_data = test_data_generator
    y_true = test_data.classes
    y_pred_prob = model.predict(test_data)
    y_pred = np.round(y_pred_prob).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    print("accuracy : ",accuracy)
    cm = confusion_matrix(y_true, y_pred)
    print("confusion matrix : ", cm)
    precision = precision_score(y_true, y_pred)
    print("precision : ",precision)
    recall = recall_score(y_true, y_pred)
    print("recall : ",recall)


    f1 = f1_score(y_true, y_pred)
    print("f1 score : ",f1)

    return accuracy, cm, precision, recall, f1



load_model_evaluate("/content/drive/MyDrive/resNet50TEST-OUTPUT.h5",test_batches)

11/11 [==============================] - 4s 184ms/step
accuracy :  0.5588235294117647
confusion matrix :  [[27 21]
 [24 30]]
precision :  0.5882352941176471
recall :  0.5555555555555556
f1 score :  0.5714285714285715


(0.5588235294117647,
 array([[27, 21],
        [24, 30]]),
 0.5882352941176471,
 0.5555555555555556,
 0.5714285714285715)

In [ ]:
#prepare dataset-2 "COVID-19 Digital X-rays Forgery Dataset" "https://www.kaggle.com/datasets/nourmahmoud/covid19-digital-xrays-forgery-dataset?resource=download"
train_path ='/content/drive/MyDrive/KaggleRealandFakeCovid19XRAYDetection/train'
valid_path ='/content/drive/MyDrive/KaggleRealandFakeCovid19XRAYDetection/valid'
test_path ='/content/drive/MyDrive/KaggleRealandFakeCovid19XRAYDetection/test'


train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input).flow_from_directory(test_path, target_size=(224,224), batch_size=10)


Found 3200 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [ ]:
# Training module using ResNet50

input_tensor = tf.keras.layers.Input(shape=(224,224,3))

base_model = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)

x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)

predictions = Dense(1, activation='sigmoid')(x)

model1 = Model(inputs=base_model.input, outputs=predictions)

opt = SGD(lr=1e-4, momentum=0.9) #SGD: It stands for Stochastic Gradient Descent, which is the optimization algorithm being used.
#lr=1e-4: It specifies the learning rate for the optimizer. The learning rate determines the step size at which the optimizer adjusts the model's parameters during training. In this case, the learning rate is set to 10^(-4), which is a small value indicating cautious and slower updates.
#momentum=0.9: Momentum is a hyperparameter that affects the convergence speed and behavior of the optimizer. It helps accelerate gradient descent in the relevant direction and dampens oscillations. A momentum value of 0.9 means that the optimizer takes into account 90% of the previously accumulated gradients to influence the current update.


callback_list=[EarlyStopping(monitor="val_loss",patience=100),ModelCheckpoint(filepath="/content/drive/MyDrive/resNet50TEST-OUTPUT.h5",monitor="val_loss",save_best_only=True,verbose=1)]


model1.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
#model1.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", false_acceptance_rate, false_rejection_rate, half_total_error_rate])
model1.summary()


history = model1.fit_generator(train_batches,validation_data=valid_batches,epochs=30,verbose=1,callbacks=callback_list)
print("Number of CNN layers:", len(base_model.layers))

#STORING HISTORY OF TRAINING FOR LATER USE

with open("history.json", "w") as f:
    json.dump(history.history, f)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

<ipython-input-8-91017a53fcce>:36: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model1.fit_generator(train_batches,validation_data=valid_batches,epochs=30,verbose=1,callbacks=callback_list)


Epoch 1/30
320/320 [==============================] - ETA: 0s - loss: 0.7432 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.69317, saving model to /content/drive/MyDrive/resNet50TEST-OUTPUT.h5
320/320 [==============================] - 1249s 4s/step - loss: 0.7432 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/30
320/320 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5000
Epoch 2: val_loss improved from 0.69317 to 0.69316, saving model to /content/drive/MyDrive/resNet50TEST-OUTPUT.h5
320/320 [==============================] - 40s 126ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/30
320/320 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5000
Epoch 3: val_loss improved from 0.69316 to 0.69316, saving model to /content/drive/MyDrive/resNet50TEST-OUTPUT.h5
320/320 [==============================] - 36s 111ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6

In [ ]:
# Testing module - using Resnet50 architecture

import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

def load_model_evaluate(model_path, test_data_generator):
    # Load the model
    model = tf.keras.models.load_model(model_path)

    # Generate predictions
    test_data = test_data_generator
    y_true = test_data.classes
    y_pred_prob = model.predict(test_data)
    y_pred = np.round(y_pred_prob).flatten()

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    print("accuracy : ",accuracy)
    cm = confusion_matrix(y_true, y_pred)
    print("confusion matrix : ", cm)
    precision = precision_score(y_true, y_pred)
    print("precision : ",precision)
    recall = recall_score(y_true, y_pred)
    print("recall : ",recall)


    f1 = f1_score(y_true, y_pred)
    print("f1 score : ",f1)

    return accuracy, cm, precision, recall, f1



load_model_evaluate("/content/drive/MyDrive/resNet50TEST-OUTPUT.h5",test_batches)

20/20 [==============================] - 2s 55ms/step
accuracy :  0.53
confusion matrix :  [[63 37]
 [57 43]]
precision :  0.5375
recall :  0.43
f1 score :  0.47777777777777775


(0.53,
 array([[63, 37],
        [57, 43]]),
 0.5375,
 0.43,
 0.47777777777777775)